Trying to use sympy as a replacement for mathematica to get my ODEs, then solving hte geodesic equaiton numerically


In [55]:
from sympy import Function, dsolve, Derivative, checkodesol,Symbol,diff, symbols
from sympy.abc import x
from sympy import MatrixSymbol, Matrix, Array
import sympy.diffgeom as dg
from sympy.diffgeom import Manifold, Patch, TensorProduct
import numpy as np
import sympy as sym
from sympy import simplify

m=Manifold('M',3)
p=Patch('P',m)



t,phi,r = symbols('t phi r')
coords=[t,phi,r]

BL=dg.CoordSystem('Boyer-Lindquist',p,[t,phi,r])



s=Symbol('s')
a=Symbol('a')



In [56]:
def Delta(r):
    delta=r**2 + a**2-r*s
    return delta

def P(r):
    p=E*(r**2 + a**2) - a*L
    return p

def R(r):
    R=-Delta(r)*((L-a*E)**2+m**2)+P(r)**2
    return R

In [57]:
tt=-(1-s/r)
tphi=-a*s/r
phiphi=r**2+a**2+a**2*s/r
rr=r**2/Delta(r)

metric_components = Matrix([[tt,tphi,0],[tphi,phiphi,0],[0,0,rr]])
(dt,dphi,dr)=BL.base_oneforms()

metric = tt*TensorProduct(dt,dt) +tphi*TensorProduct(dt,dphi)+ tphi*TensorProduct(dphi,dt)+phiphi*TensorProduct(dphi,dphi)+rr*TensorProduct(dr,dr)


print(metric_components)

print(dg.twoform_to_matrix(metric))
metric

Matrix([[-1 + s/r, -a*s/r, 0], [-a*s/r, a**2 + a**2*s/r + r**2, 0], [0, 0, r**2/(a**2 + r**2 - r*s)]])
Matrix([[-1 + s/r, -a*s/r, 0], [-a*s/r, a**2 + a**2*s/r + r**2, 0], [0, 0, r**2/(a**2 + r**2 - r*s)]])


-a*s*TensorProduct(dt, dphi)/r - a*s*TensorProduct(dphi, dt)/r + r**2*TensorProduct(dr, dr)/(a**2 + r**2 - r*s) + (-1 + s/r)*TensorProduct(dt, dt) + (a**2 + a**2*s/r + r**2)*TensorProduct(dphi, dphi)

That's encouraging

In [58]:
christ=dg.metric_to_Christoffel_2nd(metric)
#this shit do not be working
#seems to be fundamentally broken, so I'll do it myself



In [59]:
from sympy.tensor.array import MutableDenseNDimArray
def christoffel_single(metric_matrix,coords,index1,index2,index3):
    dim=len(coords)
    partialsum=0
    for beta in range(dim):
        partialsum += 1/2*metric_matrix.inv()[index1,beta]*(diff(metric_matrix[beta,index2],coords[index3])+diff(metric_matrix[beta,index3],coords[index2])-diff(metric_matrix[index2,index3],coords[beta]))
    christf_single=partialsum
    return christf_single

def christoffel_symbols(metric,coords):
    dim=len(coords)
    christoffel_total=MutableDenseNDimArray.zeros(dim,dim,dim)
    
    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                christoffel_total[i,j,k]=christoffel_single(metric,coords,i,j,k)
    return christoffel_total
    
    
    


In [60]:
christoff_pretty=simplify(christoffel_symbols(metric_components,coords))
christoff_pretty

[[[0, 0, 0.5*s*(a**2 + r**2)/(r**2*(a**2 + r**2 - r*s))], [0, 0, -0.5*a*s*(a**2 + 3*r**2)/(r**2*(a**2 + r**2 - r*s))], [0.5*s*(a**2 + r**2)/(r**2*(a**2 + r**2 - r*s)), -0.5*a*s*(a**2 + 3*r**2)/(r**2*(a**2 + r**2 - r*s)), 0]], [[0, 0, 0.5*a*s/(r**2*(a**2 + r**2 - r*s))], [0, 0, 1.0*(-0.5*a**2*s + 1.0*r**3 - 1.0*r**2*s)/(r**2*(a**2 + r**2 - r*s))], [0.5*a*s/(r**2*(a**2 + r**2 - r*s)), 1.0*(-0.5*a**2*s + 1.0*r**3 - 1.0*r**2*s)/(r**2*(a**2 + r**2 - r*s)), 0]], [[0.5*s*(a**2 + r**2 - r*s)/r**4, 0.5*a*s*(-a**2 - r**2 + r*s)/r**4, 0], [0.5*a*s*(-a**2 - r**2 + r*s)/r**4, 0.5*(a**2*s - 2*r**3)*(a**2 + r**2 - r*s)/r**4, 0], [0, 0, 1.0*(1.0*a**2 - 0.5*r*s)/(r*(a**2 + r**2 - r*s))]]]

IT FINALLY WORKED
